## Install the moules
- pip install requests (!pip install requrests)
- pip install bs4 (!pip install bs4)

In [30]:
import requests
import re
from bs4 import BeautifulSoup
import csv
import time
from pyquery import PyQuery

#### Requests 库

**requests两种访问方法** ,两者都返回Response对象：

|requests常用函数|参数解读|
|:---|:---|
|**requests.get(url, params, verify)** |发起get访问，返回**Response对象**；除非需要传参，否则不需要用**params和verify参数**| 
|**requests.post(url, data)** |发起post访问，返回**Response对象**；除非需要传参，否则不需要用**data参数**| 


##### 1.1 使用requests请求访问，得到响应
- 2开头表示访问正常
- 4开头，比如403表示爬虫被网站封锁
- 5开头表示服务器出问题

In [16]:
url = 'http://www.ynu.edu.cn/'
req = requests.get(url)  #极少数情况会用到verify=False这参数
req

<Response [200]>

### 1.2 Response响应对象

|**Response对象的方法**|作用|
|:---|:---|
|**Response.json()**|获得json格式网页数据|
|**Response.text**|获得html网页数据|
|**Response.content**|获得网页**二进制文件数据**，常常用该方法爬取**图片、视频**等数据|
|**Response.encoding**|更改网页数据的编码方式。除非使用Response.text得到的html中文本是乱码的，否则一般不用Response.encoding|
|**Response.status_code**|查看当前访问的状态码， ``200`` 表示访问正常， ``4开头`` 的状态码表示访问出问题， ``5开头`` 的状态码表示服务器出问题|

以上三种方法，最常用的是 ``text和json方法``， 分别对应于 ``html网页数据和json网页数据``

#### 1.2.1 得到html网页数据

比如我们要获取``云南大学首页的html数据``，需要使用``Response.text``方法。

In [19]:
req.text

'ï»¿<!DOCTYPE html><html lang="en"><head>\r\n\r\n\r\n\r\n    <meta charset="UTF-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">\r\n    <title>äº\x91å\x8d\x97å¤§å\xad¦</title><META Name="keywords" Content="äº\x91å\x8d\x97å¤§å\xad¦" />\r\n\r\n    <link href="css/main.css" rel="stylesheet">\r\n    <script src="js/jquery.min.js"></script>\r\n    <script src="js/jquery.SuperSlide.js"></script>\r\n    <script src="js/idangerous.swiper.min.js"></script>\r\n    <link href="css/idangerous.swiper.css" rel="stylesheet">\r\n    <script type="text/javascript">\r\n        $(function () {\r\n            $(".sidebar-nav .close").click(function () {\r\n                $(".sidebar-nav").hide()\r\n            })\r\n            $("#toTop").click(function () {\r\n                $(\'body,html\').animate({ scrollTop: 0 }, 1000);\r\n                return false;\r\n            });\r\n            $(".sidebar-nav li").hover(function () {\r\n                $(this).find(\'.pic\').stop(

### 1.3 requests.get()参数

#### 1.3.1 headers - 将爬虫伪装成浏览器
为了防止被网站封锁，我们需要将请求头headers加进访问信息中，从而将爬虫伪装成浏览器访问。

最常见的headers只有一个**User-Agent参数**即可,**让网站误以为是Chrome浏览器在访问网页数据**。

```python
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'}

url = '要访问网页的url'

r  = requests.get(url, headers = headers)
```

In [24]:
url_dianping = "http://www.dianping.com/hangzhou/hotel"
request = requests.get(url_dianping)
request

<Response [403]>

In [27]:
url_dianping = "http://www.dianping.com/hangzhou/hotel"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'}
request = requests.get(url_dianping,headers=headers)
request.text

'<!DOCTYPE html>\n<html>\n    <head>\n        <meta charset="UTF-8"/>\n        <meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n\n        <link rel="Shortcut Icon" href="//www.dpfile.com/s/res/favicon.5ff777c11d7833e57e01c9d192b7e427.ico"\n              type="image/x-icon"/>\n        <link rel="stylesheet" href="//s0.meituan.net/bs/hfe/dp-hotel-fe-pc/latest/static/css/normalize.css">\n        <link rel="stylesheet" href="//s0.meituan.net/bs/hfe/dp-hotel-fe-pc/latest/list.75e9bc0f0bca813466c4.css">\n\n        <link rel="preload" href="//s0.meituan.net/bs/hfe/dp-hotel-fe-pc/latest/list-client.75e9bc0f0bca813466c4.js" as="script">\n        <link rel="preload" href="//s0.meituan.net/bs/hfe/dp-hotel-fe-pc/latest/commons.e199aa22f4fe15a69412.js" as="script">\n\n        \n\n        <title>杭州酒店,宾馆预订-杭州酒店预订-大众点评网</title>\n        <meta name="Keywords" content=杭州酒店预订-杭州宾馆预订>\n        <meta name="Description" content=为您找到杭州1000家酒店商户信息.点击查看更多关于杭州酒店商户电话、地址、价格、评价、排行榜等详情>\n        <link rel="s

#### 简单爬虫
1. 在浏览器中打开网站的某个页面
2. 打开开发者工具，对该网页进行刷新（查看Network信息，一般都会先点击”第二页“或者”查看更多“，试图发现规律，找到真正的网址）
3. 如果是html网页，那么就用Element进行页面数据定位；如果是json数据，就在network中进行数据定位
4. 存储数据
5. 让程序自动化访问多个页面，重复1-4

#### 2. 大众点评杭州站酒店

网址模板

```
"http://www.dianping.com/hangzhou/hotel/p{num}".format(num=2) 第二页
```

爬虫从逻辑上看，包括访问、解析、存储。

先小后大，即能在一个网址上成功运行，就能在其他所有的网址（满足同一网址模板）上运行

In [32]:
#新建文件保存数据
csvf = open('\hotel.csv', 'a+', encoding='gbk', newline='')
writer = csv.writer(csvf)
writer.writerow(('hotel_name', 'place', 'price', 'review_num'))

base = "http://www.dianping.com/hangzhou/hotel/p{num}"
for page in range(1, 2):
    #网址规律
    url = base.format(num=page)
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"}
    #请求
    req = requests.get(url, headers=headers)

    #解析定位数据
    bsObj = BeautifulSoup(req.text, 'html.parser')
    hotels = bsObj.find_all('li', {"class":"hotel-block"})
    doc = PyQuery(req.text)
    for hotel in doc.items('li .hotel-info-ctn'):
        name = hotel('.hotel-name-link').text()
        place = hotel('.place').text()
        price = hotel('.price strong').text()
        review_num = hotel('.item-rank-ctn a').text()
        
        writer.writerow((name, place, price, review_num))
        print(name, place, price, review_num)
    
    time.sleep(1)
    

csvf.close()

杭州开元森泊度假酒店 杭州乐园及湘湖景区 ，距离杭州南站8.0km  
杭州桐庐康莱德酒店 红灯笼外婆家景区（桐庐） ，1km内无地铁站  
杭州湘湖逍遥庄园 杭州乐园及湘湖景区 ，距离湘湖地铁站1.1km  
杭州西湖国宾馆 西湖风景区/灵隐寺 ，距离西湖国宾馆110m  
杭州康莱德酒店 钱江新城 ，距离江锦路地铁站200m  
杭州柏悦酒店 钱江新城 ，距离钱江路地铁站290m  
杭州西子湖四季酒店 西湖风景区/灵隐寺 ，距离曲院风荷500m  
杭州临安湍口众安氡温泉度假酒店 清凉峰镇 ，1km内无地铁站  
杭州溪中溪度假酒店 西湖风景区/灵隐寺 ，距离杭州城站8.1km  
杭州西溪悦榕庄 西溪湿地 ，距离西溪天堂385m  
千岛湖洲际度假酒店 东南湖区 ，1km内无地铁站  
杭州JW万豪酒店 武林广场 ，距离武林门地铁站585m  
杭州西溪十里芳菲主题度假村落 西溪湿地 ，距离杭州城站11.8km  
杭州远洋凯宾斯基酒店 运河商务区 ，距离欧尚超市815m  
杭州洲际酒店 钱江新城 ，距离市民中心255m  
